# Advent of Code 2021
## [Day 19: Beacon Scanner](https://adventofcode.com/2021/day/19)

#### Load Data

In [1]:
import numpy as np
import re
from scipy.special import binom
from itertools import combinations, permutations, product

In [2]:
def split_input_sections(input_lines):
    current_section = []
    sections = [current_section]
    for line in input_lines:
        line = line.strip()
        if len(line) > 0:
            current_section.append(line)
        else:
            current_section = []
            sections.append(current_section)
    return sections

In [3]:
import aocd
input_data = aocd.get_data(year=2021, day=19).split('\n')
input_data[:5]

['--- scanner 0 ---',
 '878,-830,-633',
 '-654,579,583',
 '-297,520,-671',
 '-288,418,-607']

In [4]:
def parse_section(section):
    nums = [line.split(',') for line in section[1:]]
    return np.array(nums, dtype=int)
def parse_data(input_data):
    sections = split_input_sections(input_data)
    return [parse_section(s) for s in sections]
scanners = parse_data(input_data)
scanners[0][:5]

array([[ 878, -830, -633],
       [-654,  579,  583],
       [-297,  520, -671],
       [-288,  418, -607],
       [ 927, -707, -599]])

- want to find pairs of scanners with at least 12 beacons overlapping.

- plotting all beacons as a sparse bitmap would take gigabytes

- a rotation matrix could operate directly on (x,y,z) points

- then we need to compare each pair of scanners
    - for each orientation
    - for each starting beacon
    
- or should we convert the list of beacons to a list of distances?

In [5]:
scanners[0].shape

(25, 3)

In [6]:
binom(25, 2)

300.0

In [7]:
def get_dist(p1, p2):
    (x1, y1, z1), (x2, y2, z2) = p1, p2
    return (x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2
    
def get_dists(scanner):
    dists = set()
    for pair in combinations(scanner, 2):
        dists.add(get_dist(*pair))
    return dists

In [8]:
get_dists(scanners[0]).intersection(get_dists(scanners[3]))

{15038,
 22914,
 24974,
 35174,
 39986,
 53434,
 1548937,
 1562285,
 1710261,
 1780411,
 1832507,
 1844869,
 1847513,
 1969821,
 1996933}

^_^

In [9]:
overlaps = np.zeros((len(scanners), len(scanners)), dtype=int)
overlap_pairs = []

for i in range(len(scanners)-1):
    dists_i = get_dists(scanners[i])
    overlaps[i,i] = binom(len(scanners[i]), 2)
    for j in range(i+1, len(scanners)):
        dists_j = get_dists(scanners[j])
        overlap = dists_i.intersection(dists_j)
        overlaps[i,j] = len(overlap)
        overlaps[j,i] = len(overlap)
        if len(overlap) >= binom(12,2):
            overlap_pairs.append((i,j))
            #print(f"overlap between scanners {i} and {j}: {len(overlap)}")


np.set_printoptions(linewidth=110)
print(overlaps)

[[300   0   0  15   0   1   0   4   0  66   0   0   0   1   0   0   0  66   0   3  15   0  15   0  15  16]
 [  0 300   0   0   0   1   0   0  66   0   0   0   0   0   1   0  15   0  15   0   0  66   0   1   0   0]
 [  0   0 300   0   0   0  66   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [ 15   0   0 325   0   1   0   0   0  66   0  15   0   0   0  66   0   3   0  66   0   0   0   0  15   0]
 [  0   0   0   0 325   0   0  15   0   0   0   0  66  66   0   0   0   0   0   0   0   0   0  15   0   3]
 [  1   1   0   1   0 325   1   0   1   1   0   0   0   0  66   0   0   0   0   0   0   0   0   3   0   0]
 [  0   0  66   0   0   1 325   0   0   0   1  66   0   0   0  15   1   0   0   0   0  15   0   0   0   0]
 [  4   0   0   0  15   0   0 325   0  15  15   0   4  66  15   0   0  15   0   0  66   0   0  15  66  66]
 [  0  66   0   0   0   1   0   0 300   0   0   0   0   0   1   0   0   0  66   0   0   0   0   1   0   0]
 [ 66   0   0  66   0   1   0  15   0

In [10]:
def check_overlap(i, j):
    s1 = scanners[i]
    s2 = scanners[j]
    
    s1_dists = {}
    for pair in combinations(s1, 2):
        s1_dists[get_dist(*pair)] = pair

    s2_dists = {}
    for pair in combinations(s2, 2):
        s2_dists[get_dist(*pair)] = pair
        
    for d1 in s1_dists.keys():
        if d1 in s2_dists:
            #print(s1_dists[d1], s2_dists[d1])
            pass

check_overlap(*overlap_pairs[0])

In [11]:
scanners[0][0]

array([ 878, -830, -633])

In [12]:
np.array([
    [1, 0, 0],
    [0, 0, 1],
    [0, 1, 0]
]) @ scanners[0][0]

array([ 878, -633, -830])

In [13]:
rotations = np.array([np.array(x) for x in permutations(np.eye(3,3, dtype=int))])
len(rotations)

6

In [14]:
reflections = [np.roll([-1,1,1], i) for i in range(3)]
reflections = [[1, 1, 1], [-1, 1, 1], [1, -1, 1], [1, 1, -1]]
reflections

[[1, 1, 1], [-1, 1, 1], [1, -1, 1], [1, 1, -1]]

In [15]:
transforms = [rot * ref for rot, ref in product(rotations, reflections)]
len(transforms)

24

In [16]:
scanners[0][0] @ transforms

array([[ 878, -830, -633],
       [-878, -830, -633],
       [ 878,  830, -633],
       [ 878, -830,  633],
       [ 878, -633, -830],
       [-878, -633, -830],
       [ 878,  633, -830],
       [ 878, -633,  830],
       [-830,  878, -633],
       [ 830,  878, -633],
       [-830, -878, -633],
       [-830,  878,  633],
       [-633,  878, -830],
       [ 633,  878, -830],
       [-633, -878, -830],
       [-633,  878,  830],
       [-830, -633,  878],
       [ 830, -633,  878],
       [-830,  633,  878],
       [-830, -633, -878],
       [-633, -830,  878],
       [ 633, -830,  878],
       [-633,  830,  878],
       [-633, -830, -878]])

In [17]:
my_t = np.vstack([
    np.hstack([transforms[0], -scanners[0][0].reshape(-1, 1)]),
    [0, 0, 0, 1]
])
my_t

array([[   1,    0,    0, -878],
       [   0,    1,    0,  830],
       [   0,    0,    1,  633],
       [   0,    0,    0,    1]])

In [18]:
my_t @ np.hstack([scanners[0][0], 1])

array([0, 0, 0, 1])

☞ Now, the goal is to find each scanner's transform relative to scanner 0.

set scanners[0].tf to identity
compute scanners[0].distances

for each scanner without a tf:
    select some scanner with a tf and at least binom(12) matching distances
    

set other scanners' tf to None
set cursor to scanners[0]
for each scanner adjacent to cursor, if it has tf None:


In [19]:
import functools

def get_dist(p1, p2):
    (x1, y1, z1), (x2, y2, z2) = p1, p2
    return (x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2
    
def get_dists(scanner):
    dists = set()
    for pair in combinations(scanner, 2):
        dists.add(get_dist(*pair))
    return dists

class Scanner(object):
    def __init__(self, input_section):
        self.id = int(re.search(r'scanner (\d+)', input_section[0])[1])
        self.rot = None
        self.loc = None
        self.beacons = np.array([line.split(',') for line in input_section[1:]], dtype=int)    
        self.dists = get_dists(self.beacons)
        
    @functools.cached_property
    def dists(self):
        return get_dists(self.beacons)
        
    def __getitem__(self, i) -> np.array:
        return self.beacons[i]
    
    def __repr__(self) -> str:
        if self.rot is None or self.loc is None:
            tf = "Not located"
        else:
            tf = "Located"
        return f"<Scanner{self.id: 3d}: {len(self.beacons)} beacons. {tf}.>"
    
    def __and__(self, other: "Scanner") -> set:
        # usage: overlap_dists = scanner1 & scanner2
        return self.dists.intersection(other.dists)
    
    def overlaps(self, other: "Scanner") -> bool:
        return (len(self & other) >= 66)
    
    def transformed_beacons(self) -> np.array:
        return self.beacons @ self.rot - self.loc

scanners = [Scanner(s) for s in split_input_sections(input_data)]
scanners[0].rot = transforms[0]
scanners[0].loc = np.zeros(3, dtype=int)
scanners[0]

<Scanner  0: 25 beacons. Located.>

In [20]:
scanners = [Scanner(s) for s in split_input_sections(input_data)]
scanners[0].tf = np.eye(4,4)
scanners[0].overlaps(scanners[9])

True

In [21]:
scanners[0].transformed_beacons()

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [ ]:
scanners[0][:]